In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
from shapely import wkb
from shapely import errors
import itertools
import matplotlib.pyplot as plt
import datetime as dt
import holidays
import math

from Resources import ml_utils as mlu

In [2]:
weather_df = pd.read_csv('../Resources/NewYorkCity_Temperature')

In [3]:
zone_long_lat_data = pd.read_parquet('https://data.source.coop/cholmes/nyc-taxi-zones/taxi_zones_4326.parquet')

In [4]:
jan_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
feb_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')
mar_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet')
apr_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-04.parquet')
may_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-05.parquet')
jun_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-06.parquet')
jul_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-07.parquet')
aug_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-08.parquet')
sep_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-09.parquet')
oct_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet')
nov_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-11.parquet')
dec_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-12.parquet')

In [5]:
dfs = [jan_df, feb_df, mar_df, apr_df, may_df, jun_df, jul_df, aug_df, sep_df, oct_df, nov_df, dec_df]

ny_taxi_2024_df = pd.concat(dfs)

In [6]:
jan_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-01.parquet').sample(frac=0.035, random_state=1)
feb_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-02.parquet').sample(frac=0.035, random_state=1)
mar_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-03.parquet').sample(frac=0.035, random_state=1)
apr_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-04.parquet').sample(frac=0.035, random_state=1)
may_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-05.parquet').sample(frac=0.035, random_state=1)
jun_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-06.parquet').sample(frac=0.035, random_state=1)
jul_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-07.parquet').sample(frac=0.035, random_state=1)
aug_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-08.parquet').sample(frac=0.035, random_state=1)
sep_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-09.parquet').sample(frac=0.035, random_state=1)
oct_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-10.parquet').sample(frac=0.035, random_state=1)
nov_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-11.parquet').sample(frac=0.035, random_state=1)
dec_app_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-12.parquet').sample(frac=0.035, random_state=1)

In [7]:
fhv_dfs = [jan_app_df, feb_app_df, mar_app_df, apr_app_df, mar_app_df, jun_app_df, jul_app_df, aug_app_df, sep_app_df, oct_app_df, nov_app_df, dec_app_df]
fhv_2024_df = pd.concat(fhv_dfs)

In [8]:
ny_taxi_2024_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [9]:
results = mlu.impute_all(ny_taxi_2024_df, fhv_2024_df, zone_long_lat_data, weather_df)

results

c:\Users\Nik\Desktop\project_2\ny_ride_fair_optimization\nik_folder\Resources\ml_utils.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fhv_2024_df['trip_distance'] = round(fhv_2024_df['trip_distance'], 2)
c:\Users\Nik\Desktop\project_2\ny_ride_fair_optimization\nik_folder\Resources\ml_utils.py:147: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 16.11  14.3   22.36 ...  53.6   56.68 147.16]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  fhv_2024_df.loc[calc, 'total_amount'] = sum_of_columns[calc]


,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,prcp,temp,PUx,PUy,...,PU_Bronx,PU_Brooklyn,PU_Manhattan,PU_Queens,PU_Staten Island,DO_Bronx,DO_Brooklyn,DO_Manhattan,DO_Queens,DO_Staten Island
0,5,1,False,1,0,0,0.0,42.80,-8.213879e+06,4.960411e+06,...,False,False,False,True,False,False,False,True,False,False
1,9,1,False,1,0,0,0.0,42.80,-8.220657e+06,4.992372e+06,...,True,False,False,False,False,True,False,False,False,False
2,33,1,False,1,0,0,0.0,42.80,-8.231953e+06,4.974857e+06,...,False,False,False,True,False,False,True,False,False,False
3,38,1,False,1,0,0,0.0,42.80,-8.212420e+06,4.977409e+06,...,False,False,False,True,False,False,False,False,True,False
4,42,1,False,1,0,0,0.0,42.80,-8.235990e+06,4.975327e+06,...,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044442,85846,366,False,0,0,0,0.0,48.02,-8.217649e+06,4.988984e+06,...,True,False,False,False,False,True,False,False,False,False
6044443,85864,366,False,0,0,0,0.0,48.02,-8.234034e+06,4.964909e+06,...,False,True,False,False,False,False,True,False,False,False
6044444,85870,366,False,0,0,0,0.0,48.02,-8.236077e+06,4.972248e+06,...,False,False,True,False,False,False,False,True,False,False
6044445,86042,366,False,0,0,0,0.0,48.02,-8.220615e+06,4.964609e+06,...,False,False,False,True,False,False,True,False,False,False


In [10]:
results.columns

MultiIndex([(   'second_of_day',),
            (     'day_of_year',),
            (         'weekend',),
            (         'holiday',),
            (    'morning_rush',),
            (    'evening rush',),
            (            'prcp',),
            (            'temp',),
            (             'PUx',),
            (             'PUy',),
            (             'DOx',),
            (             'DOy',),
            (        'distance',),
            (   'duration(sec)',),
            (            'fare',),
            (           'tolls',),
            (         'airport',),
            (      'congestion',),
            (           'total',),
            (           'class',),
            (        'PU_Bronx',),
            (     'PU_Brooklyn',),
            (    'PU_Manhattan',),
            (       'PU_Queens',),
            ('PU_Staten Island',),
            (        'DO_Bronx',),
            (     'DO_Brooklyn',),
            (    'DO_Manhattan',),
            (       

In [11]:
results.to_parquet('Resources/model_df.parquet')